# AGI C# SDK - Quickstart
This notebook demonstrates how to use the AGI C# SDK to create and control remote browser agents.

## Setup
Install the NuGet package and import the SDK.

In [ ]:
#r "nuget: Agi, 0.5.0"

using Agi;
using Agi.Types;

Console.WriteLine("AGI SDK loaded");

## Create a Client
Initialize the client with your API key.

In [ ]:
// Uses AGI_API_KEY environment variable by default
var client = new AgiClient();
Console.WriteLine("Connected to AGI API");

## Create a Session
Create a remote browser session using the `agi-0` agent.

In [ ]:
var session = await client.Sessions.CreateAsync("agi-0");
Console.WriteLine($"Session ID: {session.SessionId}");
Console.WriteLine($"Status: {session.Status}");

## Send a Task
Send a task to the agent and stream the results in real-time.

In [ ]:
await client.Sessions.SendMessageAsync(session.SessionId, "What is the current time? Search Google to find out.");

await foreach (var evt in client.Sessions.StreamEventsAsync(session.SessionId))
{
    var eventType = evt.Event.ToString().ToLowerInvariant();
    var content = "";
    
    try
    {
        var data = evt.Data;
        if (data.ValueKind == System.Text.Json.JsonValueKind.Object)
        {
            if (data.TryGetProperty("content", out var c)) content = c.ToString();
            else if (data.TryGetProperty("message", out var m)) content = m.ToString();
            else if (data.TryGetProperty("text", out var t)) content = t.ToString();
            else content = data.ToString();
        }
        else content = data.ToString();
    }
    catch { content = evt.Data.ToString(); }

    switch (eventType)
    {
        case "thought":
            Console.WriteLine($"\U0001f4ad {(content.Length > 100 ? content[..100] : content)}");
            break;
        case "step":
            Console.WriteLine($"\u25b6\ufe0f {(content.Length > 100 ? content[..100] : content)}");
            break;
        case "done":
            Console.WriteLine($"\u2705 Done: {(content.Length > 200 ? content[..200] : content)}");
            break;
        case "error":
            Console.WriteLine($"\u274c Error: {content}");
            break;
        case "question":
            Console.WriteLine($"\u2753 Agent asks: {content}");
            break;
    }
    
    if (eventType is "done" or "error") break;
}

## Take a Screenshot
Capture the current state of the remote browser.

In [ ]:
var screenshot = await client.Sessions.ScreenshotAsync(session.SessionId);
var imgData = screenshot.Data;
if (imgData.Contains(","))
    imgData = imgData.Split(",")[1];

var bytes = Convert.FromBase64String(imgData);
Console.WriteLine($"Screenshot captured ({bytes.Length} bytes)");

## Session Control
Pause, resume, or cancel the agent.

In [ ]:
await client.Sessions.PauseAsync(session.SessionId);
Console.WriteLine("Session paused");

await client.Sessions.ResumeAsync(session.SessionId);
Console.WriteLine("Session resumed");

## Cleanup
Delete the session when done.

In [ ]:
await client.Sessions.DeleteAsync(session.SessionId);
Console.WriteLine("Session deleted");

## Using the High-Level API
The recommended way - handles cleanup automatically with IAsyncDisposable.

In [ ]:
await using var managedSession = await client.SessionAsync("agi-0");
var result = await managedSession.RunTaskAsync(
    "Search for the latest .NET release version on dotnet.microsoft.com");
Console.WriteLine($"Result: {result.Data}");
Console.WriteLine($"Duration: {result.Metadata.Duration}s");
Console.WriteLine($"Steps: {result.Metadata.Steps}");